In [15]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split

In [16]:
# Load the ISEAR dataset
df = pd.read_csv('ISEAR.csv', encoding='latin-1', names=['Emotion', 'Text'])
df = df[['Text', 'Emotion']]
df = df.dropna()

In [17]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000, oov_token=True)
tokenizer.fit_on_texts(df['Text'])

# Convert the text data to sequences
X = tokenizer.texts_to_sequences(df['Text'])
X = pad_sequences(X, padding='post')

# Convert the emotion labels to numerical values
y = pd.get_dummies(df['Emotion']).values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
# Define the BiLSTM model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [22]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [23]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
143/143 [==============================] - 38s 33ms/step - loss: 1.4007 - accuracy: 0.3157 - val_loss: 1.1488 - val_accuracy: 0.5101
Epoch 2/20
143/143 [==============================] - 3s 24ms/step - loss: 0.6723 - accuracy: 0.7672 - val_loss: 0.5584 - val_accuracy: 0.8179
Epoch 3/20
143/143 [==============================] - 3s 24ms/step - loss: 0.2646 - accuracy: 0.9276 - val_loss: 0.5113 - val_accuracy: 0.8399
Epoch 4/20
143/143 [==============================] - 3s 24ms/step - loss: 0.1581 - accuracy: 0.9595 - val_loss: 0.5325 - val_accuracy: 0.8417
Epoch 5/20
143/143 [==============================] - 3s 24ms/step - loss: 0.1215 - accuracy: 0.9679 - val_loss: 0.5093 - val_accuracy: 0.8540
Epoch 6/20
143/143 [==============================] - 3s 24ms/step - loss: 0.1060 - accuracy: 0.9716 - val_loss: 0.5451 - val_accuracy: 0.8531
Epoch 7/20
143/143 [==============================] - 3s 24ms/step - loss: 0.0987 - accuracy: 0.9685 - val_loss: 0.5407 - val_accuracy: 0.851

In [24]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9536104202270508
Test accuracy: 0.8233638405799866
